## POINTWISE MUTUAL INFORMATION
The idea of PMI is that we want to quantify the likelihood of co-occurrence of two words, taking into account the fact that it might be caused by the frequency of the single words. Hence, the algorithm computes the (log) probability of co-occurrence scaled by the product of the single probability of occurrence.
Now, knowing that, when ‘a’ and ‘b’ are independent, their joint probability is equal to the product of their marginal probabilities, when the ratio equals 1 (hence the log equals 0), it means that the two words together don’t form a unique concept: they co-occur by chance.
On the other hand, if either one of the words (or even both of them) has a low probability of occurrence if singularly considered, but its joint probability together with the other word is high, it means that the two are likely to express a unique concept.

In [1]:
import spacy
import string
import re
import pandas as pd
import numpy as np
import seaborn as sns
import random
import pickle
from unidecode import unidecode
import nltk
nltk.download('wordnet')
nltk.download('words')
nltk.download('punkt')

from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.util import skipgrams
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords 
from spacy.lang.en import English

from tqdm import tqdm
from gensim import corpora
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.models import CoherenceModel
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures,TrigramCollocationFinder, TrigramAssocMeasures

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## PMI - DRUG

In [3]:
#Reload save pkl file
with open('drug.pkl', 'rb') as f:
    drug = pickle.load(f)

In [4]:
def get_pmi_value(word1, word2, matrix):
    return matrix.loc[matrix.index == word1, [word2]]

def pmi(df, positive=True):
    cols = df.sum(axis=0)
    total = cols.sum()
    rows = df.sum(axis=1)
    expected = np.outer(rows, cols) / total
    df_pmi = df / expected
    with np.errstate(divide='ignore'):
        df_pmi = np.log(df_pmi)
    df_pmi[np.array(np.isinf(df_pmi))] = 0.0 
    if positive:
        df_pmi[df_pmi < 0] = 0.0
    return df_pmi

def max_cooccurrences(df_column):
    return df_column.idxmax() , df_column.max()

In [5]:
vectorizer = CountVectorizer(min_df=0.01,  ngram_range=(3,3))
X = vectorizer.fit_transform(drug.Text)

In [6]:
pmi_drug = pd.DataFrame(X.todense(), columns=vectorizer.vocabulary_)

In [7]:
asint = pmi_drug.astype(int)
drug_pmi = asint.T.dot(asint)

In [8]:
drug_pmi.values[[np.arange(drug_pmi.shape[0])]*2] = 0

<ipython-input-8-66051e55d8d7>:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  drug_pmi.values[[np.arange(drug_pmi.shape[0])]*2] = 0


In [9]:
drug_pmi

,guilty reasonable doubt,evidence sufficient sustain,sufficient sustain conviction,evidence favorable prosecution,favorable prosecution rational,prosecution rational essential,guilt reasonable doubt,reasonable doubt guilt,evidence closely balanced,contrary manifest evidence,...,recover damages sustained,unreasonable improbable unsatisfactory,murder armed robbery,doubt guilt evidence,evidence tending prove,ascertain effect intent,evidence aggravation mitigation,shall shall shall,conclusion foregoing affirm,guilty armed robbery
guilty reasonable doubt,0,1,1,0,0,0,7,3,0,3,...,0,1,0,1,2,2,0,2,6,1
evidence sufficient sustain,1,0,0,4,5,4,6,0,2,6,...,0,5,9,2,0,1,0,1,0,0
sufficient sustain conviction,1,0,0,5,2,0,15,19,0,2,...,1,1,1,0,0,0,0,0,30,26
evidence favorable prosecution,0,4,5,0,7,6,91,25,0,88,...,7,1,0,0,1,9,0,0,80,9
favorable prosecution rational,0,5,2,7,0,0,5,3,4,18,...,6,1,0,3,3,8,4,5,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ascertain effect intent,2,1,0,9,8,11,12,0,6,9,...,10,2,0,12,21,0,7,10,0,2
evidence aggravation mitigation,0,0,0,0,4,5,0,1,4,1,...,6,0,1,23,5,7,0,95,0,0
shall shall shall,2,1,0,0,5,7,2,1,56,1,...,9,0,0,48,5,10,95,0,2,0
conclusion foregoing affirm,6,0,30,80,7,22,89,105,0,3,...,24,5,1,0,0,0,0,2,0,86


In [10]:
drug_occ = pmi(drug_pmi, positive=True)
drug_occ

,guilty reasonable doubt,evidence sufficient sustain,sufficient sustain conviction,evidence favorable prosecution,favorable prosecution rational,prosecution rational essential,guilt reasonable doubt,reasonable doubt guilt,evidence closely balanced,contrary manifest evidence,...,recover damages sustained,unreasonable improbable unsatisfactory,murder armed robbery,doubt guilt evidence,evidence tending prove,ascertain effect intent,evidence aggravation mitigation,shall shall shall,conclusion foregoing affirm,guilty armed robbery
guilty reasonable doubt,0.000000,0.875149,0.664160,0.000000,0.000000,0.000000,0.911269,0.565069,0.000000,0.549603,...,0.000000,1.401759,0.000000,0.000000,1.206267,0.295599,0.000000,0.000000,0.956685,0.000000
evidence sufficient sustain,0.875149,0.000000,0.000000,0.642616,1.551458,1.399430,0.416611,0.000000,0.000000,0.902243,...,0.000000,2.670690,3.940324,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sufficient sustain conviction,0.664160,0.000000,0.000000,0.654770,0.424178,0.000000,1.121912,1.859399,0.000000,0.000000,...,0.000000,0.850262,1.532110,0.000000,0.000000,0.000000,0.000000,0.000000,2.014626,2.632452
evidence favorable prosecution,0.000000,0.642616,0.654770,0.000000,0.609610,0.526575,1.857390,1.066505,0.000000,2.309500,...,0.264692,0.000000,0.000000,0.000000,0.000000,0.180849,0.000000,0.000000,1.928124,0.504249
favorable prosecution rational,0.000000,1.551458,0.424178,0.609610,0.000000,0.000000,0.000000,0.000000,0.000000,1.408233,...,0.796240,0.468630,0.000000,0.000000,0.678603,0.748765,0.000000,0.000000,0.177706,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ascertain effect intent,0.295599,0.000000,0.000000,0.180849,0.748765,1.138334,0.000000,0.000000,0.000000,0.035011,...,0.626990,0.481702,0.000000,0.000000,1.944438,0.000000,0.000000,0.000000,0.000000,0.000000
evidence aggravation mitigation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.142325,0.029646,0.000000,0.000000,1.056613,0.000000,0.000000
shall shall shall,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.715658,0.000000,...,0.000000,0.000000,0.000000,0.350934,0.000000,0.000000,1.056613,0.000000,0.000000,0.000000
conclusion foregoing affirm,0.956685,0.000000,2.014626,1.928124,0.177706,1.393954,1.403263,2.069686,0.000000,0.000000,...,1.064932,0.960465,0.032875,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.329468


In [11]:
max_cooccurrences(drug_occ)

(guilty reasonable doubt                      reversed absent discretion
 evidence sufficient sustain                        murder armed robbery
 sufficient sustain conviction                   tested positive cocaine
 evidence favorable prosecution          determine credibility testimony
 favorable prosecution rational              evidence sufficient sustain
                                                    ...                 
 ascertain effect intent            improbable unsatisfactory reasonable
 evidence aggravation mitigation         evidence insufficient establish
 shall shall shall                       evidence insufficient establish
 conclusion foregoing affirm                        guilty armed robbery
 guilty armed robbery                        reasonable degree certainty
 Length: 83, dtype: object,
 guilty reasonable doubt            1.875406
 evidence sufficient sustain        3.940324
 sufficient sustain conviction      4.592488
 evidence favorable prosecution   

## PMI - WEAPONS

In [12]:
#Reload save pkl file
with open('weapons.pkl', 'rb') as f:
    weapons = pickle.load(f)

In [13]:
vectorizer = CountVectorizer(min_df=0.01, ngram_range=(3,3))
X = vectorizer.fit_transform(weapons.Text)

In [14]:
pmi_weapons = pd.DataFrame(X.todense(), columns=vectorizer.vocabulary_)

In [15]:
asint = pmi_weapons.astype(int)
weapons_pmi = asint.T.dot(asint)

In [16]:
weapons_pmi.values[[np.arange(weapons_pmi.shape[0])]*2] = 0

<ipython-input-16-a9ed64a53ec1>:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  weapons_pmi.values[[np.arange(weapons_pmi.shape[0])]*2] = 0


In [17]:
weapons_occ = pmi(weapons_pmi, positive=True)
weapons_occ

,guilty degree murder,proven guilty reasonable,guilty reasonable doubt,prove guilty reasonable,evidence favorable rational,rational essential reasonable,essential reasonable doubt,reasonable doubt evidence,unsatisfactory reasonable doubt,reasonable doubt guilt,...,testimony testimony evidence,bodily forcible felony,conclusion foregoing affirm,armed violence statute,evidence tending prove,verdict manifest evidence,violence armed violence,felony armed violence,robbery armed weapon,shall provided shall
guilty degree murder,0.000000,0.323992,0.753101,0.000000,0.299945,0.000000,0.000000,0.186317,0.147465,0.000000,...,0.000000,0.000000,0.697357,0.000000,0.428398,0.0,0.000000,0.000000,0.000000,1.193021
proven guilty reasonable,0.323992,0.000000,0.000000,0.000000,0.017314,0.083018,1.095503,0.000000,0.298470,0.000000,...,0.477352,0.623675,0.560680,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.816630
guilty reasonable doubt,0.753101,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.574836,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.652571,0.191890,0.876344,0.000000
prove guilty reasonable,0.000000,0.000000,0.000000,0.000000,0.000000,0.079446,0.000000,0.002392,0.000000,0.000000,...,0.000000,0.907784,0.000000,0.198232,0.729981,0.0,0.000000,0.552439,0.000000,0.000000
evidence favorable rational,0.299945,0.017314,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.047840,0.481845,0.000000,0.273068,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
verdict manifest evidence,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.118439,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.726848,0.0,0.000000,0.000000,0.000000,0.000000
violence armed violence,0.000000,0.000000,0.652571,0.000000,0.000000,0.000000,0.000000,0.197636,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.870686,2.083020,0.000000
felony armed violence,0.000000,0.000000,0.191890,0.552439,0.000000,0.000000,0.000000,0.295735,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.870686,0.000000,2.147840,0.000000
robbery armed weapon,0.000000,0.000000,0.876344,0.000000,0.000000,0.000000,0.000000,0.464025,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2.083020,2.147840,0.000000,0.000000


In [18]:
max_cooccurrences(weapons_occ)

(guilty degree murder                 evidence sufficient prove
 proven guilty reasonable           disturbed absent discretion
 guilty reasonable doubt         sufficient preclude imposition
 prove guilty reasonable               degree murder conviction
 evidence favorable rational     conviction evidence improbable
                                             ...               
 verdict manifest evidence             degree murder conviction
 violence armed violence            reasonable doubt conviction
 felony armed violence          evidence aggravation mitigation
 robbery armed weapon           evidence aggravation mitigation
 shall provided shall                        guilty plea guilty
 Length: 250, dtype: object,
 guilty degree murder           2.740966
 proven guilty reasonable       1.857319
 guilty reasonable doubt        3.055922
 prove guilty reasonable        3.055473
 evidence favorable rational    2.954884
                                  ...   
 verdict manifest evi

## PMI - FINANCE

In [19]:
#Reload save pkl file
with open('finance.pkl', 'rb') as f:
    finance = pickle.load(f)

In [20]:
vectorizer = CountVectorizer(min_df=0.01,  ngram_range=(3,3))
X = vectorizer.fit_transform(finance.Text)

In [21]:
pmi_finance= pd.DataFrame(X.todense(), columns=vectorizer.vocabulary_)

In [22]:
asint = pmi_finance.astype(int)
finance_pmi = asint.T.dot(asint)

In [23]:
finance_pmi.values[[np.arange(finance_pmi.shape[0])]*2] = 0

<ipython-input-23-82fdcd6bc2ed>:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  finance_pmi.values[[np.arange(finance_pmi.shape[0])]*2] = 0


In [24]:
finance_pmi

,contrary manifest evidence,guilty reasonable doubt,evidence evidence evidence,breach fiduciary duty,shall shall shall,shall provided shall,provided shall shall,accordance expressed reversed,brought recover damages,evidence tending prove,evidence reversed reversed
contrary manifest evidence,0,1,1,8,6,0,1,1,6,13,21
guilty reasonable doubt,1,0,9,173,1,1,0,72,11,1,18
evidence evidence evidence,1,9,0,10,1,9,23,2,8,4,8
breach fiduciary duty,8,173,10,0,32,12,3,11,13,5,14
shall shall shall,6,1,1,32,0,6,14,28,8,6,11
shall provided shall,0,1,9,12,6,0,11,4,2,1,2
provided shall shall,1,0,23,3,14,11,0,5,3,5,11
accordance expressed reversed,1,72,2,11,28,4,5,0,0,2,0
brought recover damages,6,11,8,13,8,2,3,0,0,101,190
evidence tending prove,13,1,4,5,6,1,5,2,101,0,120


In [25]:
finance_occ = pmi(finance_pmi, positive=True)
finance_occ

,contrary manifest evidence,guilty reasonable doubt,evidence evidence evidence,breach fiduciary duty,shall shall shall,shall provided shall,provided shall shall,accordance expressed reversed,brought recover damages,evidence tending prove,evidence reversed reversed
contrary manifest evidence,0.000000,0.000000,0.000000,0.010134,0.633419,0.000000,0.000000,0.000000,0.000000,0.581037,0.634684
guilty reasonable doubt,0.000000,0.000000,0.000000,1.484945,0.000000,0.000000,0.000000,1.418360,0.000000,0.000000,0.000000
evidence evidence evidence,0.000000,0.000000,0.000000,0.000000,0.000000,1.638025,2.116763,0.000000,0.000000,0.000000,0.000000
breach fiduciary duty,0.010134,1.484945,0.000000,0.000000,0.729483,0.604841,0.000000,0.000000,0.000000,0.000000,0.000000
shall shall shall,0.633419,0.000000,0.000000,0.729483,0.000000,0.822661,1.210426,1.405993,0.000000,0.000000,0.000000
shall provided shall,0.000000,0.000000,1.638025,0.604841,0.822661,0.000000,1.825451,0.316270,0.000000,0.000000,0.000000
provided shall shall,0.000000,0.000000,2.116763,0.000000,1.210426,1.825451,0.000000,0.079881,0.000000,0.000000,0.000000
accordance expressed reversed,0.000000,1.418360,0.000000,0.000000,1.405993,0.316270,0.079881,0.000000,0.000000,0.000000,0.000000
brought recover damages,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.856840,1.062817
evidence tending prove,0.581037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.856840,0.000000,0.885136


In [26]:
max_cooccurrences(finance_pmi)

(contrary manifest evidence       evidence reversed reversed
 guilty reasonable doubt               breach fiduciary duty
 evidence evidence evidence             provided shall shall
 breach fiduciary duty               guilty reasonable doubt
 shall shall shall                     breach fiduciary duty
 shall provided shall                  breach fiduciary duty
 provided shall shall             evidence evidence evidence
 accordance expressed reversed       guilty reasonable doubt
 brought recover damages          evidence reversed reversed
 evidence tending prove           evidence reversed reversed
 evidence reversed reversed          brought recover damages
 dtype: object,
 contrary manifest evidence        21
 guilty reasonable doubt          173
 evidence evidence evidence        23
 breach fiduciary duty            173
 shall shall shall                 32
 shall provided shall              12
 provided shall shall              23
 accordance expressed reversed     72
 brought 

## PMI - SEXUAL

In [27]:
#Reload save pkl file
with open('sexual.pkl', 'rb') as f:
    sexual = pickle.load(f)

In [28]:
vectorizer = CountVectorizer(min_df=0.01 ,ngram_range=(3,3))
X = vectorizer.fit_transform(sexual.Text)

In [29]:
pmi= pd.DataFrame(X.todense(), columns=vectorizer.vocabulary_)

In [30]:
asint = pmi.astype(int)
_pmi = asint.T.dot(asint)

In [31]:
_pmi.values[[np.arange(_pmi.shape[0])]*2] = 0

<ipython-input-31-6327eb31c154>:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  _pmi.values[[np.arange(_pmi.shape[0])]*2] = 0


In [32]:
_pmi

,manifest evidence decision,contrary manifest evidence,evidence manifest evidence,manifest evidence opposite,evidence opposite conclusion,opposite conclusion clearly,brought recover damages,foregoing reversed reversed,duty exercise reasonable,duty breach duty,...,provided shall shall,recover damages sustained,shall shall shall,decision contrary manifest,conclusion clearly evident,accordance expressed reversed,verdict notwithstanding verdict,evidence tending prove,shall provided shall,evidence guilty negligence
manifest evidence decision,0,1,0,0,0,1,1,0,0,3,...,0,1,0,4,1,0,4,2,2,1
contrary manifest evidence,1,0,4,1,1,6,0,1,8,0,...,3,0,0,1,0,4,2,1,0,0
evidence manifest evidence,0,4,0,4,15,69,21,0,10,24,...,15,23,0,0,1,3,0,6,17,4
manifest evidence opposite,0,1,4,0,4,10,7,0,0,18,...,2,8,5,0,111,3,2,1,22,13
evidence opposite conclusion,0,1,15,4,0,65,17,9,61,5,...,102,14,3,1,1,1,0,0,41,8
opposite conclusion clearly,1,6,69,10,65,0,55,350,258,21,...,104,60,65,1,5,11,1,7,89,58
brought recover damages,1,0,21,7,17,55,0,0,6,50,...,18,121,2,1,5,1,2,0,47,37
foregoing reversed reversed,0,1,0,0,9,350,0,0,92,0,...,18,0,16,1,0,3,0,1,1,0
duty exercise reasonable,0,8,10,0,61,258,6,92,0,7,...,80,5,50,7,0,13,4,3,6,3
duty breach duty,3,0,24,18,5,21,50,0,7,0,...,6,59,10,2,15,0,0,6,32,24


In [33]:
max_cooccurrences(_pmi)

(manifest evidence decision              contrary verdict evidence
 contrary manifest evidence               duty exercise reasonable
 evidence manifest evidence            opposite conclusion clearly
 manifest evidence opposite             conclusion clearly evident
 evidence opposite conclusion                 provided shall shall
 opposite conclusion clearly           foregoing reversed reversed
 brought recover damages                 recover damages sustained
 foregoing reversed reversed           opposite conclusion clearly
 duty exercise reasonable              opposite conclusion clearly
 duty breach duty                        recover damages sustained
 duty exercise ordinary                 evidence manifest evidence
 evidence directed verdict              manifest evidence evidence
 evidence evidence evidence             manifest evidence evidence
 overwhelmingly contrary verdict                  duty breach duty
 contrary verdict evidence              manifest evidence deci

## PMI - DIVORCE

In [34]:
#Reload save pkl file
with open('divorce.pkl', 'rb') as f:
    divorce = pickle.load(f)

In [35]:
vectorizer = CountVectorizer(min_df=0.01, ngram_range=(3,3))
X = vectorizer.fit_transform(divorce.Text)

In [36]:
pmi= pd.DataFrame(X.todense(), columns=vectorizer.vocabulary_)

In [37]:
asint = pmi.astype(int)
_pmi = asint.T.dot(asint)

In [38]:
_pmi.values[[np.arange(_pmi.shape[0])]*2] = 0

<ipython-input-38-6327eb31c154>:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  _pmi.values[[np.arange(_pmi.shape[0])]*2] = 0


In [39]:
_pmi

,contrary manifest evidence,shall fit reasonable,divorce extreme repeated,extreme repeated cruelty,decision manifest evidence,divorce divorce divorce,reversed consistent reversed,provided shall shall,accordance expressed reversed,guilty extreme repeated,shall shall shall,evidence evidence evidence,brought recover damages,reversed enter accordance,evidence tending prove
contrary manifest evidence,0,1,3,0,5,3,0,0,6,0,1,0,25,2,4
shall fit reasonable,1,0,0,0,1,1,2,1,1,0,0,1,0,0,0
divorce extreme repeated,3,0,0,40,7,8,5,5,28,9,1,5,0,6,0
extreme repeated cruelty,0,0,40,0,0,1,4,0,2,0,0,0,0,0,0
decision manifest evidence,5,1,7,0,0,1,1,0,11,4,1,0,1,6,1
divorce divorce divorce,3,1,8,1,1,0,2,2,244,26,0,2,3,1,0
reversed consistent reversed,0,2,5,4,1,2,0,1,9,3,0,0,4,0,0
provided shall shall,0,1,5,0,0,2,1,0,5,0,0,0,0,0,0
accordance expressed reversed,6,1,28,2,11,244,9,5,0,253,0,3,8,4,5
guilty extreme repeated,0,0,9,0,4,26,3,0,253,0,0,1,0,0,1


In [40]:
max_cooccurrences(_pmi)

(contrary manifest evidence             brought recover damages
 shall fit reasonable              reversed consistent reversed
 divorce extreme repeated              extreme repeated cruelty
 extreme repeated cruelty              divorce extreme repeated
 decision manifest evidence       accordance expressed reversed
 divorce divorce divorce          accordance expressed reversed
 reversed consistent reversed     accordance expressed reversed
 provided shall shall                  divorce extreme repeated
 accordance expressed reversed          guilty extreme repeated
 guilty extreme repeated          accordance expressed reversed
 shall shall shall                       evidence tending prove
 evidence evidence evidence            divorce extreme repeated
 brought recover damages             contrary manifest evidence
 reversed enter accordance             divorce extreme repeated
 evidence tending prove                       shall shall shall
 dtype: object,
 contrary manifest evide

### PMI - BURGLARY

In [41]:
#Reload save pkl file
with open('burglary.pkl', 'rb') as f:
    burglary = pickle.load(f)

In [42]:
vectorizer = CountVectorizer(min_df=0.01,  ngram_range=(3,3))
X = vectorizer.fit_transform(burglary.Text)

In [43]:
pmi= pd.DataFrame(X.todense(), columns=vectorizer.vocabulary_)

In [44]:
asint = pmi.astype(int)
_pmi = asint.T.dot(asint)

In [45]:
_pmi.values[[np.arange(_pmi.shape[0])]*2] = 0

<ipython-input-45-6327eb31c154>:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  _pmi.values[[np.arange(_pmi.shape[0])]*2] = 0


In [46]:
_pmi

,contrary manifest evidence,sexual assault sexual,assault sexual assault,accordingly reversed reversed,evidence evidence evidence,plea guilty guilty,factual basis plea,evidence aggravation mitigation,prove reasonable doubt,deviate sexual assault,...,murder armed robbery,armed robbery armed,ascertain effect intent,reversed consistent reversed,minimum maximum sentence,armed robbery conviction,shall guilty misdemeanor,prove preponderance evidence,evidence sufficient sustain,evidence establish guilt
contrary manifest evidence,0,6,1,3,2,2,0,14,1,0,...,0,5,1,2,2,3,2,3,23,4
sexual assault sexual,6,0,79,51,7,10,9,9,88,2,...,5,460,17,10,0,10,6,5,61,33
assault sexual assault,1,79,0,41,3,1,7,12,333,2,...,2,75,14,2,0,12,4,5,33,20
accordingly reversed reversed,3,51,41,0,1,1,1,4,63,1,...,0,58,6,1,0,11,4,12,54,31
evidence evidence evidence,2,7,3,1,0,30,4,25,5,1,...,5,9,4,65,7,2,7,8,24,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
armed robbery conviction,3,10,12,11,2,11,6,6,24,12,...,0,6,2,24,5,0,33,43,0,7
shall guilty misdemeanor,2,6,4,4,7,21,13,6,4,0,...,0,4,1,53,0,33,0,134,0,2
prove preponderance evidence,3,5,5,12,8,49,12,15,5,2,...,1,7,1,92,4,43,134,0,1,2
evidence sufficient sustain,23,61,33,54,24,94,157,45,29,7,...,66,48,541,230,7,0,0,1,0,2785


In [47]:
max_cooccurrences(_pmi)

(contrary manifest evidence       improbable unsatisfactory reasonable
 sexual assault sexual                             armed robbery armed
 assault sexual assault                         prove reasonable doubt
 accordingly reversed reversed              reasonable doubt testimony
 evidence evidence evidence                 decision manifest evidence
                                                  ...                 
 armed robbery conviction         improbable unsatisfactory reasonable
 shall guilty misdemeanor                            enter plea guilty
 prove preponderance evidence                        enter plea guilty
 evidence sufficient sustain                  evidence establish guilt
 evidence establish guilt                  evidence sufficient sustain
 Length: 76, dtype: object,
 contrary manifest evidence         40
 sexual assault sexual             460
 assault sexual assault            333
 accordingly reversed reversed     218
 evidence evidence evidence        